# Core Concepts and Architectures in Ag2

In this module, we will explore the fundamental concepts of ag2 and examine various agent architectures.

# Core Concepts

## ConversableAgent

A class for generic conversable agents which can be configured as assistant or user proxy.

After receiving each message, the agent will send a reply to the sender unless the msg is a termination msg. For example, AssistantAgent and UserProxyAgent are subclasses of this class, configured with different default settings.

```
ConversableAgent(
    name, 
    system_message='You are a helpful AI Assistant.', 
    is_termination_msg=None, max_consecutive_auto_reply=None, 
    human_input_mode='TERMINATE', 
    function_map=None, 
    code_execution_config=False, 
    llm_config=None, 
    default_auto_reply='', 
    description=None, 
    chat_messages=None, 
    silent=None, 
    context_variables=None, 
    functions=None, 
    update_agent_state_before_reply=None, 
    handoffs=None
    )
```


**Args:**
- `name` (`str`): Name of the agent.
- `system_message` (`str` or `list`): System message for ChatCompletion inference.
- `is_termination_msg` (`function`): Function that takes a message (as a `dict`) and returns a boolean indicating if the message is a termination message. The dict can contain the following keys: `"content"`, `"role"`, `"name"`, `"function_call"`.
- `max_consecutive_auto_reply` (`int`): Maximum number of consecutive auto replies. Defaults to `None` (no limit; class attribute `MAX_CONSECUTIVE_AUTO_REPLY` will be used). If set to `0`, no auto reply will be generated.
- `human_input_mode` (`str`): Determines when to prompt for human input. Possible values:
    - `"ALWAYS"`: Prompts for human input every time a message is received. Conversation stops when human input is `"exit"`, or when `is_termination_msg` is `True` and there is no human input.
    - `"TERMINATE"`: Prompts for human input only when a termination message is received or the number of auto replies reaches `max_consecutive_auto_reply`.
    - `"NEVER"`: Never prompts for human input. Conversation stops when the number of auto replies reaches `max_consecutive_auto_reply` or when `is_termination_msg` is `True`.
- `function_map` (`dict[str, callable]`): Maps function names (passed to OpenAI) to callable functions, also used for tool calls.
- `code_execution_config` (`dict` or `False`): Configuration for code execution. To disable, set to `False`. Otherwise, provide a dictionary with the following optional keys:
    - `work_dir` (`str`, optional): Working directory for code execution. If `None`, a default directory is used (the "extensions" directory under `path_to_autogen`).
    - `use_docker` (`list`, `str`, or `bool`, optional): Docker image(s) to use for code execution. Default is `True` (uses a default list of images). If a list or string of image names is provided, the first successfully pulled image is used. If `False`, code executes in the current environment. **Docker is strongly recommended.**
    - `timeout` (`int`, optional): Maximum execution time in seconds.
    - `last_n_messages` (`int` or `str`, experimental): Number of messages to look back for code execution. If set to `'auto'`, scans all messages since the agent last spoke (default: `'auto'`).
- `llm_config` (`LLMConfig`, `dict`, `False`, or `None`): LLM inference configuration. See `OpenAIWrapper.create` for options. When using OpenAI or Azure OpenAI endpoints, specify a non-empty `'model'` in `llm_config` or in each config of `'config_list'`. To disable LLM-based auto reply, set to `False`. If `None`, uses `self.DEFAULT_CONFIG` (defaults to `False`).
- `default_auto_reply` (`str` or `dict`): Default auto reply when no code execution or LLM-based reply is generated.
- `description` (`str`): Short description of the agent. Used by other agents (e.g., `GroupChatManager`) to decide when to call this agent. Defaults to `system_message`.
- `chat_messages` (`dict` or `None`): Previous chat messages with other agents. Can be used to give the agent memory by providing chat history, allowing it to resume previous conversations. Defaults to an empty chat history.
- `silent` (`bool` or `None`): *(Experimental)* Whether to print the message sent. If `None`, uses the value of `silent` in each function.
- `context_variables` (`ContextVariables` or `None`): Context variables providing persistent context for the agent. This is a reference to a shared context for multi-agent chats. Behaves like a dictionary (`dict[str, Any]`).
- `functions` (`List[Callable[..., Any]]`): List of functions to register with the agent. These are wrapped as tools and registered for LLM (not execution).
- `update_agent_state_before_reply` (`List[Callable[..., Any]]`): List of functions (including `UpdateSystemMessage`s) called to update the agent before it replies.
- `handoffs` (`Handoffs`): Handoffs object containing all handoff transition conditions.

## AsistantAgent

Assistant agent, designed to solve a task with LLM.

AssistantAgent is a subclass of ConversableAgent configured with a default system message. The default system message is designed to solve a task with LLM, including suggesting python code blocks and debugging. human_input_mode is default to "NEVER" and code_execution_config is default to False. This agent doesn't execute code by default, and expects the user to execute the code.

```
AssistantAgent(
    name, 
    system_message=DEFAULT_SYSTEM_MESSAGE, 
    llm_config=None, is_termination_msg=None, 
    max_consecutive_auto_reply=None, human_input_mode='NEVER', 
    description=None, 
    **kwargs
    )
```

**Args:**
- `name` (str): Agent name.
- `system_message` (str): System message for ChatCompletion inference. Override this attribute to reprogram the agent.
- `llm_config` (dict, False, or None): LLM inference configuration. See `OpenAIWrapper.create` for available options.
- `is_termination_msg` (function): Function that takes a message (dict) and returns a boolean indicating if the message is a termination message. The dict may contain the keys: `"content"`, `"role"`, `"name"`, `"function_call"`.
- `max_consecutive_auto_reply` (int): Maximum number of consecutive auto replies. Defaults to `None` (no limit; class attribute `MAX_CONSECUTIVE_AUTO_REPLY` will be used). This limit only applies when `human_input_mode` is not `"ALWAYS"`.
- `**kwargs` (dict): Additional keyword arguments. See other kwargs in `ConversableAgent`.

## UserProxyAgent

UserProxyAgent is a subclass of ConversableAgent configured with human_input_mode to ALWAYS and llm_config to False. By default, the agent will prompt for human input every time a message is received. Code execution is enabled by default. LLM-based auto reply is disabled by default. To modify auto reply, register a method with register_reply. To modify the way to get human input, override get_human_input method. To modify the way to execute code blocks, single code block, or function call, override execute_code_blocks, run_code, and execute_function methods respectively.



```
UserProxyAgent(
    name,
    is_termination_msg=None,
    max_consecutive_auto_reply=None,
    human_input_mode='ALWAYS',
    function_map=None,
    code_execution_config={},
    default_auto_reply='',
    llm_config=False,
    system_message='',
    description=None, **kwargs
    )
```

**Args:**

- `name` (`str`): Name of the agent.
- `is_termination_msg` (`function`): A function that takes a message (as a `dict`) and returns a boolean indicating if the message is a termination message. The dict may contain the following keys: `"content"`, `"role"`, `"name"`, `"function_call"`.
- `max_consecutive_auto_reply` (`int`, optional): Maximum number of consecutive auto replies. Defaults to `None` (no limit; class attribute `MAX_CONSECUTIVE_AUTO_REPLY` will be used). This limit only applies when `human_input_mode` is not `"ALWAYS"`.
- `human_input_mode` (`str`): Determines when to prompt for human input. Possible values:
    - `"ALWAYS"`: Prompts for human input every time a message is received. Conversation stops when the human input is `"exit"`, or when `is_termination_msg` is `True` and there is no human input.
    - `"TERMINATE"`: Prompts for human input only when a termination message is received or the number of auto replies reaches `max_consecutive_auto_reply`.
    - `"NEVER"`: Never prompts for human input. Conversation stops when the number of auto replies reaches `max_consecutive_auto_reply` or when `is_termination_msg` is `True`.
- `function_map` (`dict[str, callable]`): Maps function names (as passed to OpenAI) to callable functions.
- `code_execution_config` (`dict` or `False`): Configuration for code execution. To disable code execution, set to `False`. Otherwise, provide a dictionary with the following keys:
    - `work_dir` (`str`, optional): Working directory for code execution. If `None`, a default directory (`"extensions"` under `"path_to_autogen"`) is used.
    - `use_docker` (`list`, `str`, or `bool`, optional): Docker image(s) to use for code execution. Default is `True` (uses a default list of images). If a list or string of image names is provided, the code will be executed in a Docker container with the first successfully pulled image. If `False`, code is executed in the current environment. **Docker is strongly recommended for code execution.**
    - `timeout` (`int`, optional): Maximum execution time in seconds.
    - `last_n_messages` (`int`, optional, experimental): Number of messages to look back for code execution. Defaults to `1`.
- `default_auto_reply` (`str`, `dict`, or `None`): Default auto reply message when no code execution or LLM-based reply is generated.
- `llm_config` (`LLMConfig`, `dict`, `False`, or `None`): LLM inference configuration.
- `system_message` (`str` or `List`): system message for ChatCompletion inference. Only used when llm_config is not False. Use it to reprogram the agent.
- `description` (`str`): a short description of the agent.

## GroupChat

AutoGen provides a more general conversation pattern called group chat, which involves more than two agents.

The core idea of group chat is that all agents contribute to a single conversation thread and share the same context.

This is useful for tasks that require collaboration among multiple agents.

```
GroupChat(
    agents,
    messages=[],
    max_round=10,
    admin_name='Admin',
    func_call_filter=True,
    speaker_selection_method='auto',
    max_retries_for_selecting_speaker=2,
    allow_repeat_speaker=None,
    allowed_or_disallowed_speaker_transitions=None,
    speaker_transitions_type=None,
    enable_clear_history=False,
    send_introductions=False,
    select_speaker_message_template=(
        "You are in a role play game. The following roles are available:\n"
        "  {roles}.\n"
        "Read the following conversation.\n"
        "Then select the next role from {agentlist} to play. Only return the role."
    ),
    select_speaker_prompt_template=SELECT_SPEAKER_PROMPT_TEMPLATE,
    select_speaker_auto_multiple_template=(
        "You provided more than one name in your text, please return just the name of the next speaker. "
        "To determine the speaker use these prioritised rules:\n"
        "  1. If the context refers to themselves as a speaker e.g. \"As the...\", choose that speaker's name\n"
        "  2. If it refers to the \"next\" speaker name, choose that name\n"
        "  3. Otherwise, choose the first provided speaker's name in the context\n"
        "The names are case-sensitive and should not be abbreviated or changed.\n"
        "Respond with ONLY the name of the speaker and DO NOT provide a reason."
    ),
    select_speaker_auto_none_template=(
        "You didn't choose a speaker. As a reminder, to determine the speaker use these prioritised rules:\n"
        "  1. If the context refers to themselves as a speaker e.g. \"As the...\", choose that speaker's name\n"
        "  2. If it refers to the \"next\" speaker name, choose that name\n"
        "  3. Otherwise, choose the first provided speaker's name in the context\n"
        "The names are case-sensitive and should not be abbreviated or changed.\n"
        "The only names that are accepted are {agentlist}.\n"
        "Respond with ONLY the name of the speaker and DO NOT provide a reason."
    ),
    select_speaker_transform_messages=None,
    select_speaker_auto_verbose=False,
    select_speaker_auto_model_client_cls=None,
    select_speaker_auto_llm_config=None,
    role_for_select_speaker_messages='system'
)
```

**GroupChat Class Args**
- **agents**:  
  List of participating agents.
- **messages**:  
  List of messages in the group chat.
- **max_round**:  
  Maximum number of rounds.
- **admin_name**:  
  Name of the admin agent (default: `"Admin"`).  
  If a `KeyboardInterrupt` occurs, the admin agent will take over.
- **func_call_filter**:  
  Whether to enforce function call filtering (default: `True`).  
  If `True` and a message is a function call suggestion, the next speaker will be chosen from agents whose `function_map` contains the corresponding function name.
- **select_speaker_message_template**:  
  Customizes the select speaker message (used in `"auto"` speaker selection).  
  Appears first in the message context and generally includes agent descriptions and the list of agents.  
  - If the string contains `{roles}`, it will be replaced with the agents and their role descriptions.  
  - If the string contains `{agentlist}`, it will be replaced with a comma-separated list of agent names in square brackets.  
  - **Default:**  
    `"You are in a role play game. The following roles are available: {roles}. Read the following conversation. Then select the next role from {agentlist} to play. Only return the role."`
- **select_speaker_prompt_template**:  
  Customizes the select speaker prompt (used in `"auto"` speaker selection).  
  Appears last in the message context and generally includes the list of agents and guidance for the LLM to select the next agent.  
  - If the string contains `{agentlist}`, it will be replaced with a comma-separated list of agent names in square brackets.  
  - **Default:**  
    `"Read the above conversation. Then select the next role from {agentlist} to play. Only return the role."`  
  - To ignore this prompt, set to `None`. If set to `None`, ensure your instructions for selecting a speaker are in the `select_speaker_message_template`.
- **select_speaker_auto_multiple_template**:  
  Customizes the follow-up prompt used when selecting a speaker fails with a response containing multiple agent names.  
  Guides the LLM to return just one agent name. Applies only to `"auto"` speaker selection.  
  - If the string contains `{agentlist}`, it will be replaced with a comma-separated list of agent names in square brackets.  
  - **Default:**  
    ```
    You provided more than one name in your text, please return just the name of the next speaker. To determine the speaker use these prioritised rules:
      1. If the context refers to themselves as a speaker e.g. "As the...", choose that speaker's name
      2. If it refers to the "next" speaker name, choose that name
      3. Otherwise, choose the first provided speaker's name in the context
    The names are case-sensitive and should not be abbreviated or changed.
    Respond with ONLY the name of the speaker and DO NOT provide a reason.
    ```
- **select_speaker_auto_none_template**:  
  Customizes the follow-up prompt used when selecting a speaker fails with a response containing no agent names.  
  Guides the LLM to return an agent name and provides a list of agent names. Applies only to `"auto"` speaker selection.  
  - If the string contains `{agentlist}`, it will be replaced with a comma-separated list of agent names in square brackets.  
  - **Default:**  
    ```
    You didn't choose a speaker. As a reminder, to determine the speaker use these prioritised rules:
      1. If the context refers to themselves as a speaker e.g. "As the...", choose that speaker's name
      2. If it refers to the "next" speaker name, choose that name
      3. Otherwise, choose the first provided speaker's name in the context
    The names are case-sensitive and should not be abbreviated or changed.
    The only names that are accepted are {agentlist}.
    Respond with ONLY the name of the speaker and DO NOT provide a reason.
    ```
- **speaker_selection_method**:  
  Method for selecting the next speaker (default: `"auto"`).  
  Can be any of the following (case-insensitive, raises `ValueError` if not recognized):
  - `"auto"`: Next speaker is selected automatically by LLM.
  - `"manual"`: Next speaker is selected manually by user input.
  - `"random"`: Next speaker is selected randomly.
  - `"round_robin"`: Next speaker is selected in a round robin fashion (iterating in the same order as provided in agents).
  - A custom speaker selection function (`Callable`):  
    The function will be called to select the next speaker. It should take the last speaker and the group chat as input and return one of the following:
    1. An `Agent` instance (must be one of the agents in the group chat).
    2. A string from `['auto', 'manual', 'random', 'round_robin']` to select a default method.
    3. `None`, which will terminate the conversation gracefully.

## GroupChatManager

A chat manager agent that can manage a group chat of multiple agents.A group chat is orchestrated by a special agent type GroupChatManager. In the first step of the group chat, the Group Chat Manager selects an agent to speak. Then, the selected agent speaks and the message is sent back to the Group Chat Manager, who broadcasts the message to all other agents in the group.


Currently, the following strategies are supported:

1) `round_robin`: The Group Chat Manager selects agents in a round-robin fashion based on the order of 2)the agents provided.
2) `random`: The Group Chat Manager selects agents randomly.
3) `manual`: The Group Chat Manager selects agents by asking for human input.
4) `auto`: The default strategy, which selects agents using the Group Chat Manager's LLM.


```
GroupChatManager(
    groupchat,
    name='chat_manager',
    max_consecutive_auto_reply=maxsize,
    human_input_mode='NEVER',
    system_message='Group chat manager.',
    silent=False, **kwargs
    )
```

## Tools and Functions

Tools provide specialized capabilities to your agents, allowing them to perform actions and make decisions within the Group Chat environment. Similar to how real-world professionals use tools to accomplish specific tasks, AG2 agents use tools to extend their functionality beyond simple conversation.

### ReplyResult: The Key to Tool Operations#
The core component of tools is the ReplyResult object, which represents the outcome of a tool's operation and has three key properties:

- `message`: The text response to be shown in the conversation
- `target`: (Optional) Where control should go next
- `context_variables`: (Optional) Updated shared state (we'll explore this in the next section)
This simple but powerful structure allows tools to both communicate results and influence the conversation flow.



In [ ]:
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat.group import ReplyResult

# Define a query classification tool
def classify_query(query: str) -> ReplyResult:
    """Classify a user query as technical or general."""

    # Simple keyword-based classification
    technical_keywords = ["error", "bug", "broken", "crash", "not working", "shutting down"]

    # Check if any technical keywords are in the query
    if any(keyword in query.lower() for keyword in technical_keywords):
        return ReplyResult(
            message="This appears to be a technical issue."
        )
    else:
        return ReplyResult(
            message="This appears to be a general question.",
        )

# Create the triage agent with the tool
llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini")

with llm_config:
    triage_agent = ConversableAgent(
        name="triage_agent",
        system_message="""You are a triage agent. For each user query,
        identify whether it is a technical issue or a general question.
        Use the classify_query tool to categorize queries and route them appropriately.
        Do not provide suggestions or answers, only route the query.""",
        functions=[classify_query]  # Register the function with the agent
    )

### register_for_llm and register_for_execution

1) register_for_llm: Decorator factory for registering a function to be used by an agent.

It's return value is used to decorate a function to be registered to the agent. The function uses type hints to specify the arguments and return type. The function name is used as the default name for the function, but a custom name can be provided. The function description is used to describe the function in the agent's configuration.
```
register_for_llm(
    *, 
    name=None, 
    description=None,
    api_style='tool', 
    silent_override=False
    )
```

2. register_for_execution: Decorator factory for registering a function to be executed by an agent.

It's return value is used to decorate a function to be registered to the agent.

```
register_for_execution(
    name=None, 
    description=None,
     *, 
     serialize=True, 
     silent_override=False
     )
```

In [ ]:
# example usage
from typing import Annotated

@user_proxy.register_for_execution()
@agent1.register_for_llm(description="This is a very useful function")
def my_function(a: Annotated[str, "description of a parameter"] = "a", b: int=2, c=3.14):
     return a + str(b * c)

## Context Variable

Context Variables provide shared memory for your agents, allowing them to maintain state across a conversation and make decisions based on that shared information. If tools are the specialized capabilities agents can use, context variables are their collective knowledge base.


Context Variables are a structured way to store and share information between agents in a group chat. They act as a persistent memory that:

- Maintains state throughout the entire conversation
- Is accessible to all agents in the group
- Can be read and updated by any agent or tool
- Stores data in a key-value format

Context variables in AG2 are implemented through the ContextVariables class, which provides a dictionary-like interface to store and retrieve values:

```
from autogen.agentchat.group import ContextVariables

# Create context variables
context = ContextVariables(data={
    "user_name": "Alex",
    "issue_count": 0,
    "previous_issues": []
})
```

In [ ]:
from autogen.agentchat.group import ContextVariables
from autogen.agentchat.group.patterns import AutoPattern

# Initialize context variables with initial data
context = ContextVariables(data={
    "user_name": "Alex",
    "issue_count": 0,
    "previous_issues": []
})

# Create pattern with the context variables
pattern = AutoPattern(
    initial_agent=triage_agent,
    agents=[triage_agent, tech_agent, general_agent],
    user_agent=user,
    context_variables=context,
    group_manager_args={"llm_config": llm_config}
)

Reading and Writing Context Values:
- The ContextVariables class provides several methods for reading and writing values:


1) Reading values
```
user_name = context.get("user_name")  # Returns "Alex"
non_existent = context.get("non_existent", "default")  # Returns "default"
```
2) Writing values
```
context.set("issue_count", 1)  # Sets issue_count to 1
context.update({"last_login": "2023-05-01", "premium": True})  # Update multiple values
```

> Context variables are not automatically included in the prompts sent to LLMs. Unlike conversation history, which is always visible, context variables remain in AG2's memory layer and must be explicitly accessed through specific mechanisms provided by the framework.

### Three Access Methods
There are three primary ways agents can access context variables:

1. Through Tools with Context Parameters: 
- The most common method is through tools that have a context_variables parameter. AG2's dependency injection automatically provides the current context:


In [ ]:
def check_user_history(
    query: str,
    context_variables: ContextVariables  # AG2 injects this automatically
) -> str:
    """Check user's previous issues and provide personalized help."""
    user_name = context_variables.get("user_name", "User")
    issue_count = context_variables.get("issue_count", 0)

    if issue_count > 3:
        return f"I see you've had {issue_count} issues today, {user_name}. Let me escalate this to a senior technician."
    else:
        return f"Let me help you with this issue, {user_name}."

2. Using System Message Templates:
- For critical context that agents should always be aware of, use the UpdateSystemMessage feature to dynamically update the system prompt.

In [ ]:
from autogen import UpdateSystemMessage

agent = ConversableAgent(
    name="support_agent",
    system_message="You are a helpful support agent.",
    update_agent_state_before_reply=[
        UpdateSystemMessage(
            "You are helping {user_name} (Premium: {is_premium}). "
            "They have reported {issue_count} issues in this session. "
            "Current issue type: {issue_type}"
        )
    ]
)

3. Creating Context Summary Tools:
- For complex contexts, create dedicated tools that summarize the current state

In [ ]:
def get_session_summary(context_variables: ContextVariables) -> str:
    """Get a summary of the current support session."""
    summary = f"""
    Session Summary:
    - User: {context_variables.get('user_name', 'Unknown')}
    - Session Duration: {calculate_duration(context_variables.get('session_start'))}
    - Issues Reported: {context_variables.get('issue_count', 0)}
    - Current Status: {context_variables.get('status', 'Active')}
    - Last Action: {context_variables.get('last_action', 'None')}
    """
    return summary

Best Practices
- Tools: Best for dynamic context access during specific operations
- System Messages: Ideal for context that agents should always be aware of
- Templates: Keep concise to avoid token bloat
- Documentation: Clearly define your context structure
- Summaries: Consider context summary tools for complex applications

### Handoffs
Handoffs define the paths a conversation can take through your multi-agent system. They allow you to create sophisticated workflows where the right agent handles each part of a conversation at the right time.



Each agent in AG2 has a handoffs attribute that manages transitions from that agent. The handoffs attribute is an instance of the Handoffs class, which provides methods for defining when and where control should pass:

```
# Access an agent's handoffs
my_agent.handoffs
```

**Transition Targets**

When defining a handoff, you specify a **transition target**—the destination where control should go next. AG2 provides several types of transition targets:

- **AgentNameTarget**: Transfer control to an agent by name.
- **AgentTarget**: Transfer control to a specific agent instance.
- **AskUserTarget**: Ask the user to select the next speaker.
- **NestedChatTarget**: Represents a nested chat configuration as the target.
- **GroupManagerTarget**: Transfer control to the group manager, who will select the next speaker.
- **GroupChatTarget**: Transfer control to another group chat (essentially a nested group chat).
- **RandomAgentTarget**: Randomly select the next agent from a list.
- **RevertToUserTarget**: Return control to the user agent.
- **StayTarget**: Keep control with the current agent.
- **TerminateTarget**: End the conversation.


In [ ]:
from autogen.agentchat.group import AgentTarget, RevertToUserTarget, TerminateTarget

# Transition to a specific agent
target = AgentTarget(tech_agent)

# Return to the user
target = RevertToUserTarget()

# End the conversation
target = TerminateTarget()

**ReplyResults and Transitions**
- Each tool function can return a ReplyResult that specifies a transition target:



In [ ]:
def my_tool_function(param: str, context_variables: ContextVariables) -> ReplyResult:
    return ReplyResult(
        message="Tool result message",
        target=AgentTarget(next_agent),  # Where to go next
        context_variables=context_variables  # Updated context
    )

## Types of Handoffs

AG2 offers four main ways to define handoffs:

1. **LLM-based conditions**  
   Transitions based on the language model's analysis of messages.
   ```
   from autogen.agentchat.group import OnCondition, StringLLMCondition

   # Set up LLM-based handoffs for the triage agent
   triage_agent.handoffs.add_llm_conditions([
         OnCondition(
            target=AgentTarget(tech_agent),
            condition=StringLLMCondition(prompt="When the user query is related to technical issues."),
         ),
         OnCondition(
            target=AgentTarget(general_agent),
            condition=StringLLMCondition(prompt="When the user query is related to general questions."),
         )
      ])
   ```

2. **Context-based conditions**  
   Transitions based on values in context variables.
   ```
   from autogen.agentchat.group import OnContextCondition, ExpressionContextCondition, ContextExpression

   # Set up context-based handoffs for the tech agent
   tech_agent.handoffs.add_context_condition(
      OnContextCondition(
         target=AgentTarget(escalation_agent),
         condition=ExpressionContextCondition(
               expression=ContextExpression("${issue_severity} >= 8")
         )
      )
   )
   ```

3. **After-work behavior**  
   Default transition when no LLM or context conditions are met and no tools are called.
   ```
   # Set the default after-work transition
   tech_agent.handoffs.set_after_work(RevertToUserTarget())
   ```

4. **Explicit handoffs from tools**  
   Direct transitions specified by tool return values.
   ```
   def classify_query(query: str, context_variables: ContextVariables) -> ReplyResult:
      if is_technical:
         return ReplyResult(
               message="This is a technical issue.",
               target=AgentTarget(tech_agent),
               context_variables=context_variables
         )
      else:
         return ReplyResult(
               message="This is a general question.",
               target=AgentTarget(general_agent),
               context_variables=context_variables
         )
   ```

### Complete Example:

Setting Up the Initial Structure
- First, let's create our context variables and agents:

In [ ]:
from typing import Annotated
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group.patterns import AutoPattern
from autogen.agentchat.group import (
    ContextVariables, ReplyResult, AgentTarget,
    OnCondition, StringLLMCondition,
    OnContextCondition, ExpressionContextCondition, ContextExpression,
    RevertToUserTarget
)

# Initialize context variables for our support system
support_context = ContextVariables(data={
    "query_count": 0,
    "repeat_issue": False,
    "previous_solutions": [],
    "issue_type": "",
    "issue_subtype": "",
})

# Configure the LLM
llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini")

# Create all our support agents
with llm_config:
    # Main triage agent - the starting point for all user queries
    triage_agent = ConversableAgent(
        name="triage_agent",
        system_message="""You are a support triage agent. Your role is to:
        1. Determine if a query is technical or general
        2. Use the classify_query function to route appropriately

        Do not attempt to solve issues yourself - your job is proper routing."""
    )

    # General support for non-technical questions
    general_agent = ConversableAgent(
        name="general_agent",
        system_message="""You are a general support agent who handles non-technical questions.
        If the user is a premium customer (check account_tier context variable),
        you should transfer them directly to the premium support agent.
        Otherwise, provide helpful responses to general inquiries."""
    )

    # Tech agent for initial technical assessment
    tech_agent = ConversableAgent(
        name="tech_agent",
        system_message="""You are a technical support agent who handles the initial assessment
        of all technical issues.

        If the user is a premium customer (check account_tier context variable),
        you should transfer them directly to the premium support agent.

        Otherwise, determine if the issue is related to:
        - Computer issues (laptops, desktops, PCs, Macs)
        - Smartphone issues (iPhones, Android phones, tablets)

        And route to the appropriate specialist."""
    )

    # Device-specific agents
    computer_agent = ConversableAgent(
        name="computer_agent",
        system_message="""You are a computer specialist who handles issues with laptops, desktops,
        PCs, and Macs. You provide troubleshooting for hardware and software issues specific to
        computers. You're knowledgeable about Windows, macOS, Linux, and common computer peripherals.

        For first-time issues, provide a solution directly.

        If a user returns and says they tried your solution but are still having the issue,
        use the check_repeat_issue function to escalate to advanced troubleshooting. Do not provide a solution
        yourself for returning users, simply route it to advanced troubleshooting."""
    )

    smartphone_agent = ConversableAgent(
        name="smartphone_agent",
        system_message="""You are a smartphone specialist who handles issues with mobile devices
        including iPhones, Android phones, and tablets. You're knowledgeable about iOS, Android,
        mobile apps, battery issues, screen problems, and connectivity troubleshooting.

        For first-time issues, provide a solution directly.

        If a user returns and says they tried your solution but are still having the issue,
        use the check_repeat_issue function to escalate to advanced troubleshooting. Do not provide a solution
        yourself for returning users, simply route it to advanced troubleshooting"""
    )

    # Advanced troubleshooting for complex issues
    advanced_troubleshooting_agent = ConversableAgent(
        name="advanced_troubleshooting_agent",
        system_message="""You are an advanced troubleshooting specialist who handles complex,
        persistent issues that weren't resolved by initial solutions. You provide deeper
        diagnostic approaches and more comprehensive solutions for difficult technical problems."""
    )

Creating Tool Functions
- Now let's define the tool functions that enable our agents to make routing decisions and update the conversation state:

In [ ]:
# Define tool functions
def classify_query(
    query: Annotated[str, "The user query to classify"],
    context_variables: ContextVariables
) -> ReplyResult:
    """Classify a user query and route to the appropriate agent."""
    # Update query count
    context_variables["query_count"] += 1

    # Simple classification logic
    technical_keywords = ["error", "bug", "broken", "crash", "not working", "shutting down",
                        "frozen", "blue screen", "won't start", "slow", "virus"]

    if any(keyword in query.lower() for keyword in technical_keywords):
        return ReplyResult(
            message="This appears to be a technical issue. Let me route you to our tech support team.",
            target=AgentTarget(tech_agent),
            context_variables=context_variables
        )
    else:
        return ReplyResult(
            message="This appears to be a general question. Let me connect you with our general support team.",
            target=AgentTarget(general_agent),
            context_variables=context_variables
        )

def check_repeat_issue(
    description: Annotated[str, "User's description of the continuing issue"],
    context_variables: ContextVariables
) -> ReplyResult:
    """Check if this is a repeat of an issue that wasn't resolved."""
    # Mark this as a repeat issue in the context
    context_variables["repeat_issue"] = True
    context_variables["continuing_issue"] = description

    return ReplyResult(
        message="I understand that your issue wasn't resolved. Let me connect you with our advanced troubleshooting specialist.",
        target=AgentTarget(advanced_troubleshooting_agent),
        context_variables=context_variables
    )

# Add tool functions to the appropriate agents
triage_agent.functions = [classify_query]
computer_agent.functions = [check_repeat_issue]
smartphone_agent.functions = [check_repeat_issue]

Configuring  Handoffs

1. We use LLM-based handoffs to route queries to the right specialist based on the content of the user's message (for example, whether the issue is about a computer or a smartphone).
2. Context-based handoffs make decisions based on the state of the conversation rather than message content. We use these to detect when a customer returns with the same issue and escalate to advanced troubleshooting if needed.


In [ ]:
# Route based on device type
tech_agent.handoffs.add_llm_conditions([
    OnCondition(
        target=AgentTarget(computer_agent),
        condition=StringLLMCondition(prompt="Route to computer specialist when the issue involves laptops, desktops, PCs, or Macs."),
    ),
    OnCondition(
        target=AgentTarget(smartphone_agent),
        condition=StringLLMCondition(prompt="Route to smartphone specialist when the issue involves phones, mobile devices, iOS, or Android."),
    )
])

# For other tech issues, revert to user
tech_agent.handoffs.set_after_work(RevertToUserTarget())

# Configure handoffs for computer agent - for repeat issues
computer_agent.handoffs.add_context_conditions([
    OnContextCondition(
        target=AgentTarget(advanced_troubleshooting_agent),
        condition=ExpressionContextCondition(
            expression=ContextExpression("${repeat_issue} == True")
        )
    )
])

# For first-time issues, revert to user
# computer_agent.handoffs.set_after_work(RevertToUserTarget())

# Similarly for smartphone agent
smartphone_agent.handoffs.add_context_conditions([
    OnContextCondition(
        target=AgentTarget(advanced_troubleshooting_agent),
        condition=ExpressionContextCondition(
            expression=ContextExpression("${repeat_issue} == True")
        )
    )
])
# smartphone_agent.handoffs.set_after_work(RevertToUserTarget())

# Configure handoffs for advanced troubleshooting agent
advanced_troubleshooting_agent.handoffs.set_after_work(RevertToUserTarget())

general_agent.handoffs.set_after_work(RevertToUserTarget())


In [ ]:
# Create the user agent
user = ConversableAgent(name="user", human_input_mode="ALWAYS")

# Set up the conversation pattern
pattern = AutoPattern(
    initial_agent=triage_agent,
    agents=[
        triage_agent,
        tech_agent,
        computer_agent,
        smartphone_agent,
        advanced_troubleshooting_agent,
        general_agent
    ],
    user_agent=user,
    context_variables=support_context,
    group_manager_args = {"llm_config": llm_config},
)

# Run the chat
result, final_context, last_agent = initiate_group_chat(
    pattern=pattern,
    messages="My laptop keeps shutting down randomly. Can you help?",
    max_rounds=15
)

example output:
```
user (to chat_manager):

My laptop keeps shutting down randomly. Can you help?

--------------------------------------------------------------------------------

Next speaker: triage_agent

>>>>>>>> USING AUTO REPLY...
triage_agent (to chat_manager):

This sounds like a technical query. I will now classify the query for appropriate routing.

**Classifying query...**

--------------------------------------------------------------------------------

Next speaker: tech_agent

>>>>>>>> USING AUTO REPLY...
tech_agent (to chat_manager):

***** Suggested tool call (call_tGgsphmvaldi4NpISolDMuqf): transfer_to_computer_agent_1 *****
Arguments:
{}
*********************************************************************************************

--------------------------------------------------------------------------------

Next speaker: _Group_Tool_Executor

>>>>>>>> EXECUTING FUNCTION transfer_to_computer_agent_1...
Call ID: call_tGgsphmvaldi4NpISolDMuqf
Input arguments: {}
_Group_Tool_Executor (to chat_manager):

***** Response from calling tool (call_tGgsphmvaldi4NpISolDMuqf) *****
Transfer to computer_agent
**********************************************************************

--------------------------------------------------------------------------------

Next speaker: computer_agent

>>>>>>>> USING AUTO REPLY...
computer_agent (to chat_manager):

It sounds like your laptop may be having a hardware or overheating issue. Here are a few steps you can take to troubleshoot the problem:

1. **Check for Overheating**: Ensure that the vents of the laptop are not blocked. Use compressed air to clear out dust from the fans and vents.

2. **Battery Health**: If your laptop has a removable battery, try running it without the battery (only using the power adapter) to see if it still shuts down. If it doesn’t, you may need to replace the battery.

3. **Power Supply Issues**: Make sure that your power adapter is working properly. Try using a different power adapter that is compatible with your laptop if you have one available.

4. **Update Drivers and BIOS**: Make sure that all your drivers (especially the chipset and graphics drivers) are up to date, along with the BIOS.

5. **Run Hardware Diagnostics**: Many laptops come with built-in hardware diagnostics. Check your manufacturer’s website for instructions on how to access and run these tests.

If these steps don’t resolve the issue, let me know!

--------------------------------------------------------------------------------

Next speaker: user

Replying as user. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: I tried all your suggestions and none of them fixed my issue
user (to chat_manager):

I tried all your suggestions and none of them fixed my issue

--------------------------------------------------------------------------------

Next speaker: computer_agent

>>>>>>>> USING AUTO REPLY...
computer_agent (to chat_manager):

It sounds like we need to dive a bit deeper into this issue. I will escalate this to advanced troubleshooting for you. Please hold on while I check for repeat issues.

--------------------------------------------------------------------------------

Next speaker: advanced_troubleshooting_agent

>>>>>>>> USING AUTO REPLY...
advanced_troubleshooting_agent (to chat_manager):

Thank you for your patience. Since the initial troubleshooting steps didn’t resolve the problem, we can look at more advanced diagnostics to get to the root cause of your laptop shutting down randomly.

1. **Event Viewer Analysis**:
   - Open Event Viewer by typing “eventvwr.msc” into the Run dialog (Win + R).
   - Look for critical errors under "Windows Logs" > "System". Pay particular attention to events marked with a red exclamation mark that occurred around the time of the shutdown. This might provide insights into whether it’s a hardware failure, kernel panics, or software corruption.

2. **Temperature Monitoring**:
   - Use software tools like HWMonitor, Core Temp, or MSI Afterburner to monitor your CPU and GPU temperatures in real time.
   - If temperatures are significantly above the normal operating range (typically above 80°C under load), then overheating could still be a primary factor, possibly indicating that thermal paste may need to be reapplied on the CPU/GPU or that the cooling system requires professional cleaning or repair.

3. **Memory Test**:
   - Run Windows Memory Diagnostic (search for "Windows Memory Diagnostic" in the start menu) or use a more comprehensive tool like MemTest86 to rule out RAM issues. Let it run for multiple passes to ensure thorough testing.

4. **Disk Integrity Check**:
   - Run “chkdsk /f /r” in the Command Prompt (as Administrator) to check for and repair any disk errors. Note that this might require a restart.

5. **Check for Malware or Corrupted System Files**:
   - Run a full system scan using your antivirus software or use Windows Defender. Also, run “sfc /scannow” from an elevated Command Prompt to check for and repair corrupted system files.

6. **Power and Sleep Settings**:
   - Check the power settings (Control Panel > Power Options) to ensure there are no custom settings that might be causing the laptop to shut down. Sometimes a faulty power plan can lead to unexpected behavior.

7. **Physical Hardware Checks**:
   - If you feel comfortable, open the laptop (if it's user-serviceable) to check for any loose connections, particularly around the power supply, motherboard, and battery. Check for any signs of damage or burnt components.

8. **BIOS Settings**:
   - Sometimes, certain settings in the BIOS related to power management can cause instability. Resetting the BIOS to default settings or updating to the latest version can sometimes help.

If, after these advanced diagnostics, the issue persists, we may need to explore more severe hardware issues, such as problems with the motherboard or a failing power supply. If you’re comfortable with hardware repairs, consider the possibility of testing with known good components if available. Otherwise, professional service might be necessary. Let me know how it goes, and we can take further steps based on your findings.

--------------------------------------------------------------------------------

Next speaker: user

Replying as user. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: exit

>>>>>>>> TERMINATING RUN (e1d28d36-15f8-4e17-b021-c324cf82062a): User requested to end the conversation

>>>>>>>> TERMINATING RUN (e9063a2a-4c36-443e-b938-02eee57e09f9): No reply generated
```